In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'

import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import set_random_seed
from tensorflow.python.client import device_lib
import datetime as dt
from sklearn.cluster import KMeans
from collections import Counter
import matplotlib.colors as mcolors
np.set_printoptions(threshold=np.inf)
# fix random seed for reproducibility
np.random.seed(7)
set_random_seed(7)
from scipy.spatial import distance
from numpy.random import choice
import math
from scipy import stats
from scipy import optimize
import seaborn as sns; sns.set()

In [7]:
general_saving_dir = "cross_validation/averages/general/"
if not os.path.exists(general_saving_dir):
    os.makedirs(general_saving_dir)
    
reward_oriented_saving_dir = "cross_validation/averages/reward_oriented/"
if not os.path.exists(reward_oriented_saving_dir):
    os.makedirs(reward_oriented_saving_dir)
    
no_reward_saving_dir = "cross_validation/averages/no_reward/"
if not os.path.exists(no_reward_saving_dir):
    os.makedirs(no_reward_saving_dir)

# just for creating the final results when getting all folds 

In [8]:
# # # 3.7.19
seq_length = 4
with open('pd_list_full_with_rewards_original_seq4.pkl', 'rb') as f:
    pd_list = pickle.load(f)
    
# # with open('huge_pd_with_rewards_original_seq4.pkl', 'rb') as f:
# #     huge_pd = pickle.load(f)
    
with open('huge_pd_shuffled_with_rewards_original_seq4_SHUFFLED.pkl', 'rb') as f:
    huge_pd = pickle.load(f)

In [9]:
full_orig_data = pd.read_csv('DataAllSubjectsRewards.csv')
print(full_orig_data[full_orig_data['payoff_structure']==2].user.unique().shape)
print(full_orig_data[full_orig_data['payoff_structure']==3].user.unique().shape)
print(full_orig_data[full_orig_data['payoff_structure']==4].user.unique().shape)

data_filled_NA = full_orig_data.fillna(-1)
data_noNA = full_orig_data.dropna()
# notice that here is where I choose if feeding full data with blanks, or without blanks
# without blanks:
# data_payoff2 = data_noNA[data_noNA['payoff_structure']==2]
# data_payoff3 = data_noNA[data_noNA['payoff_structure']==3]
# data_payoff4 = data_noNA[data_noNA['payoff_structure']==4]
# with blanks:
data_payoff2 = data_filled_NA[data_filled_NA['payoff_structure']==2]
data_payoff3 = data_filled_NA[data_filled_NA['payoff_structure']==3]
data_payoff4 = data_filled_NA[data_filled_NA['payoff_structure']==4]
user_ids2 = data_payoff2.user.unique()
user_ids3 = data_payoff3.user.unique()
user_ids4 = data_payoff4.user.unique()

(329,)
(312,)
(324,)


In [10]:
# load the kfolded pars
with open('cross_validation/payoff2_train_participants_5fold_list.pkl', 'rb') as f:
    payoff2_train_participants_5fold_list = pickle.load(f)

with open('cross_validation/payoff2_test_participants_5fold_list.pkl', 'rb') as f:
    payoff2_test_participants_5fold_list = pickle.load(f)

with open('cross_validation/payoff3_train_participants_5fold_list.pkl', 'rb') as f:
    payoff3_train_participants_5fold_list = pickle.load(f)

with open('cross_validation/payoff3_test_participants_5fold_list.pkl', 'rb') as f:
    payoff3_test_participants_5fold_list = pickle.load(f)

with open('cross_validation/payoff4_train_participants_5fold_list.pkl', 'rb') as f:
    payoff4_train_participants_5fold_list = pickle.load(f)
    
with open('cross_validation/payoff4_test_participants_5fold_list.pkl', 'rb') as f:
    payoff4_test_participants_5fold_list = pickle.load(f)

In [11]:
total_pars = set()
for i in range(5):
    fold_set_example = [*payoff2_test_participants_5fold_list[i] , *payoff3_test_participants_5fold_list[i] , *payoff4_test_participants_5fold_list[i]]
    total_pars.update(fold_set_example)

## prior results 

In [12]:
## Full data trained model results , over full data(aka "All" or "General")

general_dir = 'cross_validation/general_model/'
filename = 'general_model_results_5folds.pkl'
with open(os.path.join(general_dir,filename), 'rb') as f:
    general_folds_results = pickle.load(f)
# persons_true_false_predictions,persons_predictions_expr,total_actual_predictions,total_predicted_correct = results
# inquiry_results = pd.read_csv('inquiry_results_SHUFFLED_NEW.csv')

In [13]:
## Reward oriented (qlearn generated data) trained model results, over full data (aka "Reward_Oriented")

reward_oriented_dir = 'cross_validation/generated_data_model/'
filename = 'simulated_data_model_results_5folds.pkl'
with open(os.path.join(reward_oriented_dir,filename), 'rb') as f:
    reward_oriented_folds_results = pickle.load(f)
# reward_oriented_persons_true_false_predictions, reward_oriented_persons_predictions_expr, reward_oriented_total_actual_predictions, reward_oriented_total_predicted_correct = reward_oriented_results

In [14]:
# ## No Reward trained model results, over full data (aka "No_Reward")

no_reward_dir = 'cross_validation/no_reward_model/'
filename = 'no_reward_model_results_5folds.pkl'
with open(os.path.join(no_reward_dir,filename), 'rb') as f:
    no_reward_folds_results = pickle.load(f)
# no_reward_persons_true_false_predictions, no_reward_persons_predictions_expr, no_reward_total_actual_predictions, no_reward_total_predicted_correct = no_reward_results

In [15]:
# each fold is :
# persons_true_false_predictions,persons_predictions_expr,total_actual_predictions,total_predicted_correct, persons_activations = results

---------------------------------

# Accuracy averages

In [16]:
def get_users_actual_choices(all_times_compare_df_payoff):
    """
    get a DataFrame holding the participants (#) with actual choice and cluster for CERTAIN time t (all_times_compare_df_payoff index 0 is time 4 actually)
    returns : ACTUAL choices for each cluster 
    """
    clusters_choices = []
    for cluster in all_times_compare_df_payoff.clusters.unique():
        clusters_choices.append(list(all_times_compare_df_payoff[all_times_compare_df_payoff['clusters']==cluster].choices))
    return clusters_choices

In [17]:
def get_users_predicted_choices(pars_in_cluster_at_time_t:dict,time):
    """
    get a dictionary holding the participants (#) in each cluster for CERTAIN time t 
    NOTICE THAT PARS_IN_CLUSTER ARE (1-965) and prediction_mat is (0-964), hence the participant-1
    returns : predictions of choices for each cluster 
    """
    clusters_choices = []
    for cluster in pars_in_cluster_at_time_t.keys():
        choices = []
        for participant in pars_in_cluster_at_time_t[cluster]:
            choices.append(prediction_mat[participant-1,time])
        clusters_choices.append(choices)
    return clusters_choices

In [18]:
def create_corresponding_sums(payoff_cluster_sums_mat, payoff_ratios_mat):
    """
    In order to get a dataframe of the clusters sums SORTED by their corresponding ratios:
    Gets: the payoff cluster sums matrix and the payoff ratios matrix.
    Returns: a df of the sorted sums (by their original ratios) ---> ascending=True like (same as sorted_ratios_payoff_for_pcolor).
    """
    sorted_args_for_ratios_mat = pd.DataFrame(np.argsort(payoff_ratios_mat,axis=0))
    check = payoff_cluster_sums_mat.T.copy()
    res = []
    for i in range(sorted_args_for_ratios_mat.shape[1]):
        res.append(np.ravel(check[i,list(sorted_args_for_ratios_mat[i])]))
    return pd.DataFrame(np.asmatrix(res).T)

In [125]:
def extract_lstm_acc_per_person(persons_true_false_predictions):
    collect_not_minuses = []
    for pred in persons_true_false_predictions:
        if pred<0:
            continue
        else:
            collect_not_minuses.append(pred)
    return np.mean(collect_not_minuses)
#     return np.mean(persons_true_false_predictions[4:])

In [20]:
def get_true_false_predictions(person_df, participant, payoff_qlearn_person_predictions, payoff_person_real_accuracies, payoff_length_correction):
    """
    gets the person_df and the curresponding user id (participant) with the qlearning predictions (and payoff_person_real_accuracies for assertion), and returns a true/false vector of 150
    """
    person_choices = person_df.choice -1
    participant_true_false = []
    for choice, prediction in zip(person_choices, payoff_qlearn_person_predictions[participant-1-payoff_length_correction]):
        if choice<0:
            participant_true_false.append(np.nan)
        else:
            if choice == prediction:
                participant_true_false.append(1)
            else:
                participant_true_false.append(0)

    participant_true_false_np = np.asarray(participant_true_false)
    assert(np.nanmean(participant_true_false_np)==payoff_person_real_accuracies[participant-1-payoff_length_correction]),"accuracy is wrong"
    participant_true_false_np = np.nan_to_num(participant_true_false_np, -1)
    return participant_true_false_np

In [21]:
# TODO : adjust this?
def get_avg_acc_per_time(payoff_person_true_false_predictions_matrix):
    """
    gets a payoff_person_true_false_predictions_matrix -  which can be either qlearning or lstm 
    return a vector of 150, each entrance is the avg accuracy across all participants in that payoff
    # usage:
    # get_avg_acc_per_time(payoff2_persons_true_false_predictions_lstm)
    """
    payoff_all_pars_avg_acc_per_time = []
    for t in range(payoff_person_true_false_predictions_matrix.shape[1]):
        collect_not_minuses = []
        for pred in payoff_person_true_false_predictions_matrix[:,t]:
            if pred<0:
                continue
            else:
                collect_not_minuses.append(pred)
        payoff_all_pars_avg_acc_per_time.append(np.mean(collect_not_minuses))
    
    return payoff_all_pars_avg_acc_per_time

# Very important :    

# Each fold has it's own participants, but concatenating all folds one after another DOES NOT results in the true order of the participnats,
# So I can't just divide regulary ! NEED TO MAP

In [22]:
def produce_true_false_correct_order(payoff2_test_participants_5fold_list, payoff3_test_participants_5fold_list, payoff4_test_participants_5fold_list, model_folds_results):

    folded_tests_sets = []
    dfs= []
    for test_2, test_3, test_4, results in zip(payoff2_test_participants_5fold_list, payoff3_test_participants_5fold_list, payoff4_test_participants_5fold_list, model_folds_results):
        fold_set = [*test_2 , *test_3 , *test_4]
        folded_tests_sets.append(fold_set)
        general_persons_true_false_predictions, general_persons_predictions_expr,\
        general_total_actual_predictions, general_total_predicted_correct, general_persons_activations = results

        cur_df = pd.DataFrame(general_persons_true_false_predictions, index=fold_set)

        dfs.append(cur_df)
        
    return pd.concat(dfs).sort_index().to_numpy()

--------------------------------------------

# Accuracy per time (average accuracy of all participants per time)
 - the accuracy per time is the average of this time over all folds
 - producing here true_false_predictions

In [23]:
general_all_folds_true_false = produce_true_false_correct_order(payoff2_test_participants_5fold_list, payoff3_test_participants_5fold_list, payoff4_test_participants_5fold_list, general_folds_results)
        

reward_oriented_all_folds_true_false = produce_true_false_correct_order(payoff2_test_participants_5fold_list, payoff3_test_participants_5fold_list, payoff4_test_participants_5fold_list, reward_oriented_folds_results)


no_reward_all_folds_true_false = produce_true_false_correct_order(payoff2_test_participants_5fold_list, payoff3_test_participants_5fold_list, payoff4_test_participants_5fold_list, no_reward_folds_results)
        

assert(np.array_equal(general_all_folds_true_false, reward_oriented_all_folds_true_false)==False)
assert(np.array_equal(general_all_folds_true_false, no_reward_all_folds_true_false)==False) 
assert(np.array_equal(no_reward_all_folds_true_false, reward_oriented_all_folds_true_false)==False)
    
# divide General lstm true/false matrix to payoffs
general_payoff2_persons_true_false_predictions_lstm = np.asmatrix(general_all_folds_true_false[0:329])
general_payoff3_persons_true_false_predictions_lstm = np.asmatrix(general_all_folds_true_false[329:641])
general_payoff4_persons_true_false_predictions_lstm = np.asmatrix(general_all_folds_true_false[641:965])
##############################################
# divide Reward-Oriented lstm true/false matrix to payoffs
reward_oriented_payoff2_persons_true_false_predictions_lstm = np.asmatrix(reward_oriented_all_folds_true_false[0:329])
reward_oriented_payoff3_persons_true_false_predictions_lstm = np.asmatrix(reward_oriented_all_folds_true_false[329:641])
reward_oriented_payoff4_persons_true_false_predictions_lstm = np.asmatrix(reward_oriented_all_folds_true_false[641:965])
##################################################
# divide No-Reward lstm true/false matrix to payoffs
no_reward_payoff2_persons_true_false_predictions_lstm = np.asmatrix(no_reward_all_folds_true_false[0:329])
no_reward_payoff3_persons_true_false_predictions_lstm = np.asmatrix(no_reward_all_folds_true_false[329:641])
no_reward_payoff4_persons_true_false_predictions_lstm = np.asmatrix(no_reward_all_folds_true_false[641:965])


general_all_folds_all_payoffs_true_false = general_all_folds_true_false, general_payoff2_persons_true_false_predictions_lstm, general_payoff3_persons_true_false_predictions_lstm, general_payoff4_persons_true_false_predictions_lstm
with open(os.path.join(general_saving_dir, 'general_all_folds_all_payoffs_true_false.pkl'), 'wb') as f:
    pickle.dump(general_all_folds_all_payoffs_true_false, f)


reward_oriented_all_folds_all_payoffs_true_false = reward_oriented_all_folds_true_false, reward_oriented_payoff2_persons_true_false_predictions_lstm, reward_oriented_payoff3_persons_true_false_predictions_lstm, reward_oriented_payoff4_persons_true_false_predictions_lstm
with open(os.path.join(reward_oriented_saving_dir, 'reward_oriented_all_folds_all_payoffs_true_false.pkl'), 'wb') as f:
    pickle.dump(reward_oriented_all_folds_all_payoffs_true_false, f)
    

no_reward_all_folds_all_payoffs_true_false = no_reward_all_folds_true_false, no_reward_payoff2_persons_true_false_predictions_lstm, no_reward_payoff3_persons_true_false_predictions_lstm, no_reward_payoff4_persons_true_false_predictions_lstm
with open(os.path.join(no_reward_saving_dir, 'no_reward_all_folds_all_payoffs_true_false.pkl'), 'wb') as f:
    pickle.dump(no_reward_all_folds_all_payoffs_true_false, f)

--------------------------------

## Accuracy per participant
 - notice that altough we get all 965 participants, they were trained and tested with 5 different models !
 - the accuracy of each participant is the accuracy that we recieved for a certain fold - since as said, the participant was not tested in other folds as well

In [128]:
################################
# first extract all accuracies #
################################
general_all_fold_lstm_accs = []
general_payoff2_lstm_accs = []
for user_ind in user_ids2:
    cur_accs = extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1])
    general_payoff2_lstm_accs.append(cur_accs)
    general_all_fold_lstm_accs.append(cur_accs)
    
general_payoff3_lstm_accs = []
for user_ind in user_ids3:
    cur_accs = extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1])
    general_payoff3_lstm_accs.append(cur_accs)
    general_all_fold_lstm_accs.append(cur_accs)
    
general_payoff4_lstm_accs = []
for user_ind in user_ids4:
    cur_accs = extract_lstm_acc_per_person(general_all_folds_true_false[user_ind-1])
    general_payoff4_lstm_accs.append(cur_accs)
    general_all_fold_lstm_accs.append(cur_accs)

#############################

reward_oriented_all_fold_lstm_accs = []
reward_oriented_payoff2_lstm_accs = []
for user_ind in user_ids2:
    cur_accs = extract_lstm_acc_per_person(reward_oriented_all_folds_true_false[user_ind-1])
    reward_oriented_payoff2_lstm_accs.append(cur_accs)
    reward_oriented_all_fold_lstm_accs.append(cur_accs)
    
reward_oriented_payoff3_lstm_accs = []
for user_ind in user_ids3:
    cur_accs = extract_lstm_acc_per_person(reward_oriented_all_folds_true_false[user_ind-1])
    reward_oriented_payoff3_lstm_accs.append(cur_accs)
    reward_oriented_all_fold_lstm_accs.append(cur_accs)
    
reward_oriented_payoff4_lstm_accs = []
for user_ind in user_ids4:
    cur_accs = extract_lstm_acc_per_person(reward_oriented_all_folds_true_false[user_ind-1])
    reward_oriented_payoff4_lstm_accs.append(cur_accs)
    reward_oriented_all_fold_lstm_accs.append(cur_accs)
    
################################


no_reward_all_fold_lstm_accs = []
no_reward_payoff2_lstm_accs = []
for user_ind in user_ids2:
    cur_accs = extract_lstm_acc_per_person(no_reward_all_folds_true_false[user_ind-1])
    no_reward_payoff2_lstm_accs.append(cur_accs)
    no_reward_all_fold_lstm_accs.append(cur_accs)
    
no_reward_payoff3_lstm_accs = []
for user_ind in user_ids3:
    cur_accs = extract_lstm_acc_per_person(no_reward_all_folds_true_false[user_ind-1])
    no_reward_payoff3_lstm_accs.append(cur_accs)
    no_reward_all_fold_lstm_accs.append(cur_accs)
    
no_reward_payoff4_lstm_accs = []
for user_ind in user_ids4:
    cur_accs = extract_lstm_acc_per_person(no_reward_all_folds_true_false[user_ind-1])
    no_reward_payoff4_lstm_accs.append(cur_accs)
    no_reward_all_fold_lstm_accs.append(cur_accs)

        


#################
# general model #
###########################################################################
  
assert(general_payoff2_lstm_accs!=general_payoff3_lstm_accs)
assert(general_payoff2_lstm_accs!=general_payoff4_lstm_accs)
assert(general_payoff3_lstm_accs!=general_payoff4_lstm_accs)


general_all_accuracy_per_par = general_all_fold_lstm_accs, general_payoff2_lstm_accs, general_payoff3_lstm_accs, general_payoff4_lstm_accs
with open(os.path.join(general_saving_dir, 'general_all_accuracy_per_par.pkl'), 'wb') as f:
    pickle.dump(general_all_accuracy_per_par, f)
############################################################################



#########################
# reward_oriented model #
###########################################################################
   
assert(reward_oriented_payoff2_lstm_accs!=reward_oriented_payoff3_lstm_accs)
assert(reward_oriented_payoff2_lstm_accs!=reward_oriented_payoff4_lstm_accs)
assert(reward_oriented_payoff3_lstm_accs!=reward_oriented_payoff4_lstm_accs)

reward_oriented_all_accuracy_per_par = reward_oriented_all_fold_lstm_accs, reward_oriented_payoff2_lstm_accs, reward_oriented_payoff3_lstm_accs, reward_oriented_payoff4_lstm_accs
with open(os.path.join(reward_oriented_saving_dir, 'reward_oriented_all_accuracy_per_par.pkl'), 'wb') as f:
    pickle.dump(reward_oriented_all_accuracy_per_par, f)
############################################################################


#########################
# no_reward model #
###########################################################################
    
assert(no_reward_payoff2_lstm_accs!=no_reward_payoff3_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=no_reward_payoff4_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=no_reward_payoff4_lstm_accs)

no_reward_all_accuracy_per_par = no_reward_all_fold_lstm_accs, no_reward_payoff2_lstm_accs, no_reward_payoff3_lstm_accs, no_reward_payoff4_lstm_accs
with open(os.path.join(no_reward_saving_dir, 'no_reward_all_accuracy_per_par.pkl'), 'wb') as f:
    pickle.dump(no_reward_all_accuracy_per_par, f)
############################################################################

assert(no_reward_payoff2_lstm_accs!=general_payoff2_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=general_payoff3_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=general_payoff4_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=reward_oriented_payoff2_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=reward_oriented_payoff3_lstm_accs)
assert(no_reward_payoff2_lstm_accs!=reward_oriented_payoff4_lstm_accs)

assert(no_reward_payoff3_lstm_accs!=general_payoff2_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=general_payoff3_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=general_payoff4_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=reward_oriented_payoff2_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=reward_oriented_payoff3_lstm_accs)
assert(no_reward_payoff3_lstm_accs!=reward_oriented_payoff4_lstm_accs)

assert(no_reward_payoff4_lstm_accs!=general_payoff2_lstm_accs)
assert(no_reward_payoff4_lstm_accs!=general_payoff3_lstm_accs)
assert(no_reward_payoff4_lstm_accs!=general_payoff4_lstm_accs)
assert(no_reward_payoff4_lstm_accs!=reward_oriented_payoff2_lstm_accs)
assert(no_reward_payoff4_lstm_accs!=reward_oriented_payoff3_lstm_accs)
assert(no_reward_payoff4_lstm_accs!=reward_oriented_payoff4_lstm_accs)

-----------------